In [1]:
import os
import glob
import h5py
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Flatten, Dense, Activation, Reshape

import tensorflow as tf
import yolo.config as cfg
from utils.help import say

Using TensorFlow backend.


In [2]:
keras.backend.set_image_dim_ordering('th')
weights_path = 'yolo-tiny-origin.h5'
is_freeze = True
verbalise = True

In [3]:
def make_yolotiny_network(is_freeze=True):
    model = Sequential()
    model.add(Conv2D(16, (3, 3), input_shape=(3,448,448),padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(64,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(128,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(256,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(512,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(1024,(3,3), padding='same', activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(Conv2D(1024,(3,3), padding='same', activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(Conv2D(1024,(3,3), padding='same', activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dense(4096))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(1470))
    
    return model

In [4]:
def make_bottleneck_network(model):
    model.add(Dense(256, input_shape=model.output_shape))
    model.add(Dense(4096))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(1470))
    
    return model

In [5]:
model = make_yolotiny_network()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 448, 448)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 224, 224)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 224, 224)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 112, 112)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 112, 112)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 56, 56)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 56, 56)       73856     
__________

In [6]:
from yolo.training_v1 import darkeras_loss, _TRAINER
from yolo.dataset.data import shuffle

In [7]:
inp_x = model.input
net_out = model.output
sess = K.get_session()

say("Building {} loss function".format(cfg.model_name), verbalise=verbalise)
loss_ph, loss_op = darkeras_loss(net_out)
say("Building {} train optimizer".format(cfg.model_name), verbalise=verbalise)
optimizer = _TRAINER[cfg.trainer](cfg.lr)
gradients = optimizer.compute_gradients(loss_op)
train_op = optimizer.apply_gradients(gradients)

sess.run(tf.global_variables_initializer())
model.load_weights(weights_path)
say("Setting weigths : {}",format(weights_path), verbalise=verbalise)

pop_layer = model.pop() # dense_25
say("{} layer poped".format(pop_layer), verbalise=verbalise)
pop_layer = model.pop() # leakyrelu_34
say("{} layer poped".format(pop_layer), verbalise=verbalise)
pop_layer = model.pop() # dense_24
say("{} layer poped".format(pop_layer), verbalise=verbalise)
pop_layer = model.pop() # dense 23
say("{} layer poped".format(pop_layer), verbalise=verbalise)

model.summary()
print(model.output_shape)

['Building yolo-tiny loss function']
Building yolo-tiny loss
['Building yolo-tiny train optimizer']
['Setting weigths : {}', 'yolo-tiny-origin.h5']
['None layer poped']
['None layer poped']
['None layer poped']
['None layer poped']
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 448, 448)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 224, 224)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 224, 224)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 112, 112)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 112, 112)      18496     
__________________________________________

In [8]:
model = make_bottleneck_network(model)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 448, 448)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 224, 224)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 224, 224)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 112, 112)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 112, 112)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 56, 56)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 56, 56)       73856     
__________

In [9]:
batches = shuffle()
for i, (x_batch, datum) in enumerate(batches):
    train_feed_dict = {
       loss_ph[key]:datum[key] for key in loss_ph 
    }
    train_feed_dict[inp_x] = x_batch
    # print("feed_dict.keys() : ", len(train_feed_dict.keys()), train_feed_dict.keys())
    fetches = [train_op, loss_op] 
    fetched = sess.run(fetches, feed_dict=train_feed_dict)
    
    loss_val = fetched[1]
    say("step {} - loss {}".format(i, loss_val), verbalise=True)
    if i == 1:
        save_step_weigths_path = 'yolo-tiny-step{}.h5'.format(i)
        model.save_weights(save_step_weigths_path)
        print("Saved weigths : ", save_step_weigths_path)
    if i % 310 == 0:
        model.save_weights('yolo-tiny-epoch{}.h5'.format(i//310))
        say("Save weights : ", 'yolo-tiny-epoch{}.h5'.format(i//310), verbalise=verbalise)

Dataset of 9963 instance(s)
shuffle index :  [  84 2981 3162 ..., 3567 1886 4583]
['step 0 - loss 1.830634593963623']
['step 1 - loss 2.9135732650756836']
Saved weigths :  yolo-tiny-step1.h5
['step 2 - loss 1.6306806802749634']
['step 3 - loss 1.977710247039795']
['step 4 - loss 2.4008426666259766']
['step 5 - loss 2.9010069370269775']
['step 6 - loss 1.898966908454895']
['step 7 - loss 2.4896249771118164']
['step 8 - loss 1.9629135131835938']
['step 9 - loss 3.87100887298584']
['step 10 - loss 1.7466118335723877']
['step 11 - loss 3.297980308532715']
['step 12 - loss 2.2121381759643555']
['step 13 - loss 2.622051239013672']
['step 14 - loss 1.8840867280960083']
['step 15 - loss 2.4684433937072754']
['step 16 - loss 1.9220449924468994']
['step 17 - loss 1.8221871852874756']
['step 18 - loss 1.88920259475708']
['step 19 - loss 2.4026126861572266']
['step 20 - loss 2.066751480102539']
['step 21 - loss 1.9796459674835205']
['step 22 - loss 2.1538681983947754']
['step 23 - loss 2.758663654

KeyboardInterrupt: 

In [10]:
sess.close()

In [12]:
def printname(name):
    print(name)

f = h5py.File(weights_path)
f.visit(printname)
f.close()

convolution2d_1
convolution2d_1/convolution2d_1_W:0
convolution2d_1/convolution2d_1_b:0
convolution2d_2
convolution2d_2/convolution2d_2_W:0
convolution2d_2/convolution2d_2_b:0
convolution2d_3
convolution2d_3/convolution2d_3_W:0
convolution2d_3/convolution2d_3_b:0
convolution2d_4
convolution2d_4/convolution2d_4_W:0
convolution2d_4/convolution2d_4_b:0
convolution2d_5
convolution2d_5/convolution2d_5_W:0
convolution2d_5/convolution2d_5_b:0
convolution2d_6
convolution2d_6/convolution2d_6_W:0
convolution2d_6/convolution2d_6_b:0
convolution2d_7
convolution2d_7/convolution2d_7_W:0
convolution2d_7/convolution2d_7_b:0
convolution2d_8
convolution2d_8/convolution2d_8_W:0
convolution2d_8/convolution2d_8_b:0
convolution2d_9
convolution2d_9/convolution2d_9_W:0
convolution2d_9/convolution2d_9_b:0
dense_1
dense_1/dense_1_W:0
dense_1/dense_1_b:0
dense_2
dense_2/dense_2_W:0
dense_2/dense_2_b:0
dense_3
dense_3/dense_3_W:0
dense_3/dense_3_b:0
flatten_1
leakyrelu_10
maxpooling2d_1
maxpooling2d_2
maxpooling2